In [2]:
import torch
from torch import nn

In [3]:
torch.__version__

'2.1.0+cu118'

In [45]:
class ConvBlock(nn.Module):
    def __init__(self,
                in_channels,
                out_channels,
                use_activation=True,
                activation='relu',
                use_batchnorm=True,
                **kwargs,
                ):
        super().__init__()
        self.use_activation = use_activation
        self.cnn = nn.Conv2d(in_channels, out_channels, bias= not use_batchnorm, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels) if use_batchnorm else nn.Identity()
        self.act = (
            nn.LeakyReLU(0.2, inplace=True)
            if activation == 'relu'
            else nn.PReLU(num_parameters=out_channels)
        )

    def forward(self, x):
      return self.act(self.bn(self.cnn(x))) if self.use_activation else self.bn(self.cnn(x))



In [37]:
# Testing our conv block
import matplotlib.pyplot as plt

with torch.inference_mode():
    test_conv_block = ConvBlock(3, 64)
    random_image = torch.rand([1,3,4,4])

    output = test_conv_block(random_image)
    print(output[0,:3])
    print(output.shape)

tensor([[[-0.2281,  1.2712],
         [-0.1595,  0.6668]],

        [[-0.3048, -0.0020],
         [ 0.2676,  1.2666]],

        [[-0.1278,  1.3301],
         [-0.2470,  0.5439]]])
torch.Size([1, 64, 2, 2])


In [70]:

class ResidualBlock(nn.Module):
    def __init__(self,
                 in_channels,
                 ):
        super().__init__()
        self.conv1 =  ConvBlock(in_channels, in_channels, True, 'PReLU',
                                use_batchnorm=True,
                                kernel_size=3,
                                stride=1,
                                padding=1)
        self.conv2 =  ConvBlock(in_channels, in_channels, False,
                                use_batchnorm=True,
                                kernel_size=3,
                                stride=1,
                                padding=1)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(x)
        # print(f"Input shape = {x.shape}, output shape = {out.shape}")
        # print(f"Output after convulation = {out[0,0]}")
        return out + x


In [ ]:
with torch.inference_mode():
    test_residual = ResidualBlock(3)
    input = torch.rand([1,3,5,5])
    print(input[:,0])
    output = test_residual(input)
    print(output[:,0])


In [68]:
## Upsample Block

class UpsampleBlock(nn.Module):

    def __init__(self,
                 in_channels,
                 upscale_factor): # factor in 1 axis .
        super().__init__()
        self.conv = nn.Conv2d(in_channels, in_channels* upscale_factor**2, kernel_size = 3, stride = 1, padding=1)
        self.shuffle = nn.PixelShuffle(upscale_factor=upscale_factor)
        self.act = nn.PReLU(num_parameters=in_channels)

    def forward(self, x):
        # print(f"Tensor shape before Upsample = {x.shape} and after upsample it becomes = {self.shuffle(self.conv(x)).shape}")
        return self.shuffle(self.conv(x))



In [ ]:
# Testing the upsample block

with torch.inference_mode():
    upsampler = UpsampleBlock(4, 2)
    input = torch.rand([1,4,5,5])
    output= upsampler(input)
    print(f"Tensor shape before Upsample = {input.shape} and after upsample it becomes = {output.shape}")
    print(input[0,0])
    print(output[0,0])

In [83]:
# Generator Class MAJOR CLASS

class Generator(nn.Module):

    def __init__(self,
                 num_channels=64,
                 num_blocks=16,
                 upscale_factor=2):
        super().__init__()
        self.initial = ConvBlock(3, num_channels,
                                 use_activation=True, activation= "PReLU",
                                 use_batchnorm=False,
                                 kernel_size=9, padding=4, stride=1)
        self.residuals = nn.Sequential(*[ResidualBlock(num_channels) for _ in range(num_blocks)] )
        self.mid_conv = ConvBlock(num_channels, num_channels, use_activation=False, use_batchnorm=True, kernel_size=3 , stride=1, padding=1)
        self.upsample = nn.Sequential(UpsampleBlock(num_channels, upscale_factor), UpsampleBlock(num_channels, upscale_factor))
        self.final_conv = nn.Conv2d(in_channels=num_channels, out_channels=3, kernel_size=9, stride= 1, padding=4)

    def forward(self, x):
        initial_out = self.initial(x)
        out = self.residuals(initial_out)
        out = self.mid_conv(out) + initial_out
        out = self.upsample(out)
        out = self.final_conv(out)
        return torch.tanh(out)




In [84]:
# Testing our generator

with torch.inference_mode():
    gen = Generator()
    input = torch.rand([1,3,5,5])
    output = gen(input)
    print(f"Shape of input = {input.shape}, and shape of output = {output.shape} ")

Shape of input = torch.Size([1, 3, 5, 5]), and shape of output = torch.Size([1, 3, 20, 20]) 
